### sklearn LogisticRegression 사용법

In [1]:
from sklearn.linear_model import LogisticRegression

이제 LogisticRegression 모델을 생성하고, 그 안에 속성들(features)과 그 레이블(labels)을 fit 시킨다. 이렇게.

In [2]:
model = LogisticRegression()
# model.fit(features, labels)

fit() 메서드는 모델에 필요한 두 가지 변수를 전달해준다.
- 계수: model.coef_
- 절편: mode.intercept_

여기서 만약 .predict()를 사용하면 새로운 속성들을 넣었을 때 그 레이블에 속하는지 아닌지 1 또는 0으로 구성된 벡터를 반환해준다.

In [3]:
# model.predict(features)

그리고 만약 분류 결과가 아니라, 해당 레이블로 분류될 확률 값을 알고 싶다면 이렇게 .predict_proba()를 사용해주면 된다. 당연히 각 샘플에 대한 확률을 0에서 1 사이의 값으로 돌려줄거다.

In [4]:
# model.predict_proba(features)

그런데 실제로 로지스틱 회귀 알고리즘을 사용하기 전에는 반드시 데이터를 정규화(Normalization) 해줘야 한다.

아래 예제에서 직접 sklearn에서 제공하는 StandardScaler을 통해 표준화, 데이터 스케일링 하는 방법을 소개하겠다.

### sklearn LogisticRegression 실전 예제

In [7]:
# 데이터 불러오기
import pandas as pd

In [6]:
passengers = pd.read_csv('train.csv')
print(passengers.shape)
print(passengers.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

**분석에 사용할 feature 고르기** <br/>
일단 생존 여부에 중요한 영향을 미쳤을 것으로 예상하는 값들만추려보면, Sex, Age, Pclass를 생각해볼 수 있다. 여성, 어린이, 1/2/3등석 순으로 살아남을 확률이 높다고 가설을 세워본 거다.

**문자열을 숫자로 변환하기** <br/>
일단 머신러닝 라이브러리 sklearn에 넣어주기 전에 성별(Sex)은 숫자 데이터 1과 0으로 바꿔주자. 여성이 살아남을 확률이 높을 것 같으니 남성을 0, 여성을 1로 놓자.

In [8]:
# 2. 데이터 전처리하기
passengers['Sex'] = passengers['Sex'].map({'female' : 1, 'male' : 0})

**결측치 채워주기** <br/>
데이터를 보면 Age 값이 비어있는 경우가 있다. 이 경우 그냥 평균 값으로 대체해서 넣어주자.

In [10]:
passengers['Age'].fillna(value = passengers['Age'].mean(), inplace = True)

**feature 분리하기** <br/>
PClass의 경우 1등석에 탔는지, 2등석에 탔는지에 대해 각각의 feature로 만들어주기 위해 컬럼을 새로 생성해보자.

In [13]:
passengers['FirstClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 1 else 0)
passengers['SecondClass'] = passengers['Pclass'].apply(lambda x: 1 if x == 2 else 0)

In [14]:
# 데이터 셋 준비
features = passengers[['Sex', 'Age', 'FirstClass', 'SecondClass']]
survival = passengers['Survived']

### 3. 학습/평가 셋 분리하기
sklearn의 train_test_split을 사용해서 학습세트와 평가세트를 분리하자.

In [15]:
from sklearn.model_selection import train_test_split
train_features, test_features, train_labels, test_labels = train_test_split(features, survival)

### 4. 데이터 정규화(스케일링)하기
중요한 파트다. 로지스틱 회귀는 실행할 때 regularation을 사용하기 때문에 그 전에 반드시 우리가 가진 데이터를 스케일링하는 작업이 필요하다. (이 부분은 이해 안 가더라도 일단 그냥 해야 된다고 생각하자.)

sklearn이 제공하는 StandardScaler를 활용해서 손쉽게 할 수 있다. StandardScaler는 평균 0, 표준편차 1로 변환하는 방법이지만 이외에도 최소값 0, 최대값 1이 되도록 변환하는 MinMaxScaler, 중앙값(median) 0, IQR(interquartile range) 1이 되도록 변환하는 RobustScaler 등이 있다.

In [18]:
# 데이터 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.fit_transform(test_features)

fit_transform은 fit과 transform을 합친 건데, fit은 일단 각 속성(feature)마다 컬럼을 만드는 작업이라고 생각하면 된다. 이후 transform을 통해 데이터를 변형시키는 거다.
그래서 위 코드에서도 학습세트로 fit을 한 번 해주었기 때문에, 평가세트에서는 별도로 fit을 할 필요 없이 바로 transform 하면 되는 거다.

### 5. 모델 생성 및 평가하기
모델을 생성하는 방법은 아래와 같다.

In [19]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(train_features, train_labels)

LogisticRegression()

In [20]:
print(model.score(train_features, train_labels))

0.7874251497005988


In [21]:
print(model.score(test_features, test_labels))

0.7892376681614349


이번엔 각 feature들의 계수(coefficients)를 확인해 볼 차례다. 어떤 feature가 생존에 큰 영향을 주는지 확인해볼 수 있으니까.

In [22]:
print(model.coef_)

[[ 1.25745993 -0.48482425  0.96522862  0.41396266]]


아까 Sex, Age, FirstClass, SecondClass 순으로 넣었기 때문에 그 순서대로 확인해주면 된다. 성별이 1(여자)이고 일등석 탑승 여부가 중요하다는 걸 알 수 있다. 반면 나이에 대한 계수는 음수가 나오는데 나이가 많을수록 생존 확률이 낮아진다는 의미로 이해하면 되겠다.

### 5. 예측하기
이번엔 새로운 데이터를 넣어서 예측해보자. <br/>
예를 들면 아래와 같이 타이타닉 영화 실제 주인공이었던 Jack, Rose의 값을 임의로 만들고, 내 정보도 넣어봤다. (가상이니까... 1등석에 탄 걸로 해봤다.)

In [23]:
import numpy as np

Jack = np.array([0.0, 20.0, 0.0, 0.0])
Rose = np.array([1.0, 17.0, 1.0, 0.0])
ME = np.array([0.0, 32.0, 1.0, 0.0])

sample_passengers = np.array([Jack, Rose, ME])

In [24]:
sample_passengers = scaler.transform(sample_passengers)

C:\Users\82104\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [25]:
print(model.predict(sample_passengers))

[0 1 0]


In [26]:
print(model.predict_proba(sample_passengers))

[[0.87583424 0.12416576]
 [0.04682353 0.95317647]
 [0.53100596 0.46899404]]
